In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# run this cell if running remotely

!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from google.colab import drive
drive.mount('/content/drive')

import tensorflow, sentence_transformers, transformers
tensorflow.__version__, sentence_transformers.__version__, transformers.__version__

root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 61.2 MB/s 
     |████████████████████████████████| 880 kB 83.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ecf1aee7f9b7c8644c7f530a8d2b6ada1184c6c4c727bf3de892bd2febeed949
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=1101b87d1d3585261fd57ef7e694e384c285b685865225a7cc30ce2770c6f373
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words
Looking in indexes: https://pypi.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Mounted at /content/drive


In [2]:
import tensorflow, sentence_transformers, transformers
tensorflow.__version__, sentence_transformers.__version__, transformers.__version__

('2.10.0', '2.2.2', '4.22.2')

In [3]:
# run this cell if working locally
import sys, os
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.insert(0, config.root_path)

In [4]:
import tensorflow as tf

from src.encoders.context_encoder_ldabert_2 import ContextEncoderSimple
from src.dataset.ldabertv3 import LDABERT3Dataset

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np
from utils.experiments import get_experiments_json, get_experiments, save_results

## Experiment

In [11]:
experiments_config = get_experiments_json('qmsum_ldabert2_simple_test')
experiments_config_df = pd.DataFrame.from_dict(experiments_config)
experiments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,max_sentence_length,gamma,pct_data,augment_pct,bert_trainable,epochs
0,ldabert,committee,0.5,128,128,15,1,1,False,10
1,ldabert,product,0.5,128,128,15,1,1,False,10
2,ldabert,academic,0.5,128,128,15,1,1,False,10


In [12]:
experiments_config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [13]:
def get_random_hash(k):
  import random, string
  x = ''.join(random.choices(string.ascii_letters + string.digits, k=k))
  return x

In [8]:
print("initializing dataset...")
dataset = LDABERT3Dataset(dataset_type="committee",
                        pct_data=1,
                        max_seq_length=128,
                        max_segment_length=300,
                        augment_pct=0,
                        split="train",
                        artificial_segments=False)

initializing dataset...


In [9]:
print("initializing dataset...")
dataset = LDABERT3Dataset(dataset_type="product",
                        pct_data=0.5,
                        max_seq_length=128,
                        max_segment_length=300,
                        augment_pct=0,
                        split="train",
                        artificial_segments=False)

# process training dataset
print("processing dataset...")
sentences, tokenized_sentences, labels = dataset.process()

vectors_filename = '{}_{}_as-{}_msl-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments, dataset.max_segment_length)

saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

initializing dataset...
processing dataset...


2022-12-23 15:24:46.666943: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-23 15:24:46.667608: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max


In [15]:
# all
for experiment in experiments_config[0:3]:
    dataset_type = experiment['dataset_type']
    final_dropout = experiment['final_dropout']
    dense_neurons = experiment['dense_neurons']
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    gamma = experiment['gamma']
    max_sentence_length = experiment['max_sentence_length']
    bert_trainable = experiment['bert_trainable']
    epochs = experiment['epochs']
    BATCH_SIZE = 32
    print("params:", experiment)
    random_hash = get_random_hash(5)
    experiment['epochs'] = epochs

    # init model
    print("initializing model...")
    model = ContextEncoderSimple(final_dropout=final_dropout,
                            dense_neurons=dense_neurons,
                          gamma=gamma,
                          max_sentence_length=max_sentence_length,
                           bert_trainable=bert_trainable)

    # print("number of params: ", sum([np.prod(keras.get_value(w).shape) for w in model.trainable_weights]))

    # init training dataset
    print("initializing dataset...")
    dataset = LDABERT3Dataset(dataset_type=dataset_type,
                            pct_data=pct_data,
                            max_seq_length=max_sentence_length,
                            max_segment_length=300,
                            augment_pct=0,
                            split="train",
                            artificial_segments=False)

    # process training dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process()

    vectors_filename = '{}_{}_as-{}_msl-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments, dataset.max_segment_length)

    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

    if len(saved_vectors) == 0:
        saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

    left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
    lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)


    # init testing dataset
    print("initializing testing dataset...")
    testing_dataset = LDABERT3Dataset(dataset_type=dataset_type,
                            pct_data=0.25,
                          max_seq_length=512,
                            max_segment_length=300,
                            augment_pct=0,
                            split="test",
                        artificial_segments=False)

    # process testing dataset
    print("processing testing dataset...")
    testing_sentences, testing_tokenized_sentences, testing_labels = testing_dataset.process()

    vectors_filename = '{}_{}_testing_data.pkl'.format(testing_dataset.pct_data, testing_dataset.augment_pct)

    testing_saved_vectors, testing_saved_labels, testing_saved_sentences, testing_saved_tokenized_sentences = testing_dataset.get_saved_vectors("test", testing_dataset.dataset_type, vectors_filename)

    if len(testing_saved_vectors) == 0:
        testing_saved_vectors, testing_saved_labels, testing_saved_sentences, testing_saved_tokenized_sentences = testing_dataset.create_vectors("test", testing_dataset.dataset_type, vectors_filename)

    testing_left_input, testing_mid_input, testing_right_input = testing_dataset.format_sentences_tri_input_plus(testing_saved_tokenized_sentences)
    testing_lda_left_input, testing_lda_mid_input, testing_lda_right_input = testing_dataset.format_sentences_tri_input(testing_saved_vectors)

    # get class weight
    neg, pos = np.bincount(labels.flatten())
    initial_bias = np.log([pos/neg])

    total=len(labels)
    weight_for_0 = (1 / neg)*(total)/2.0
    weight_for_1 = (1 / pos)*(total)/2.0

    class_weight = {0: weight_for_0, 1: weight_for_1}
    print("class weight", class_weight)

    # create checkpoint path
    checkpoint_filepath = '{}/models/LDABERT2/simple/{}-{}-{}-pct-{}-aug_{}/no-finetune/checkpoint.ckpt'.format(
                            config.root_path,
                            dataset.dataset_type,
                            len(saved_sentences),
                            dataset.pct_data,
                            dataset.augment_pct,
                            random_hash)

    # # continue training
    # random_hash = "SwPSB"
    # checkpoint_filepath = '{}/models/LDABERT2/simple/wiki-40561-1-pct-0-aug_{}/no-finetune/checkpoint.ckpt'.format(
    #     config.root_path,
    #     random_hash
    # )

    # model_filepath = "/".join(checkpoint_filepath.split("/")[:-1]) + "/model.h5"
    print(checkpoint_filepath)

    # get callbacks ready.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        save_best_only=True,
        mode="auto",
        save_freq="epoch")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        verbose=1,
        patience=10,
        mode='max',
        restore_best_weights=True)

    log_path = '{}/models/LDABERT2/simple/{}-{}-{}-pct-{}-aug_{}/no-finetune/training.log'.format(
                            config.root_path,
                            dataset.dataset_type,
                            len(saved_sentences),
                            dataset.pct_data,
                            dataset.augment_pct,
                            random_hash)

    csv_logger = tf.keras.callbacks.CSVLogger(log_path, separator=",", append=False)

    callbacks = [
    #     early_stopping,
        model_checkpoint_callback,
        csv_logger
    ]

    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[
                      keras.metrics.BinaryAccuracy(name='accuracy')
                  ])

    try:
        model.load_weights(checkpoint_filepath)
        print("model loaded.")
    except:
        print("No checkpoint available.")

    print("starting the training process...")
    history = model.fit([
                            left_input, mid_input, right_input,
                            lda_left_input, lda_mid_input, lda_right_input
                        ],
                        tf.convert_to_tensor(saved_labels),
                        validation_data=([
                            testing_left_input, testing_mid_input, testing_right_input,
                            testing_lda_left_input, testing_lda_mid_input, testing_lda_right_input
                        ], tf.convert_to_tensor(testing_saved_labels)),
                        epochs=epochs,
                        # validation_split=0.25,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        class_weight=class_weight,
                        callbacks=callbacks)

    # assigning history to experiment object for saving.
    experiment["history"] = history.history
    experiment["hash"] = random_hash

    print("saving results...")
    save_results(experiment)

params: {'bert_type': 'ldabert', 'dataset_type': 'committee', 'final_dropout': 0.5, 'dense_neurons': 128, 'max_sentence_length': 128, 'gamma': 15, 'pct_data': 1, 'augment_pct': 1, 'bert_trainable': False, 'epochs': 150}
initializing model...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


initializing dataset...
processing dataset...
initializing testing dataset...
processing testing dataset...
class weight {0: 0.5095998640727212, 1: 26.542035398230087}
/Users/amitmaraj/Documents/PhD/context-encoder-qmsum/models/LDABERT2/simple/committee-11997-1-pct-0-aug_oMuZU/no-finetune/checkpoint.ckpt
compiling the model...
No checkpoint available.
starting the training process...
Epoch 1/150


2022-12-23 19:36:35.084858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - ETA: 0s - loss: 0.5789 - accuracy: 0.8167

2022-12-23 19:42:05.209781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 742s 2s/step - loss: 0.5789 - accuracy: 0.8167 - val_loss: 0.3532 - val_accuracy: 0.8085
Epoch 2/150
375/375 [==============================] - 581s 2s/step - loss: 0.5099 - accuracy: 0.8069 - val_loss: 0.3031 - val_accuracy: 0.8131
Epoch 3/150
375/375 [==============================] - 587s 2s/step - loss: 0.4830 - accuracy: 0.8152 - val_loss: 0.2693 - val_accuracy: 0.8204
Epoch 4/150
375/375 [==============================] - 587s 2s/step - loss: 0.4794 - accuracy: 0.8235 - val_loss: 0.3080 - val_accuracy: 0.8266
Epoch 5/150
375/375 [==============================] - 583s 2s/step - loss: 0.4589 - accuracy: 0.8268 - val_loss: 0.2354 - val_accuracy: 0.8287
Epoch 6/150
375/375 [==============================] - 583s 2s/step - loss: 0.4479 - accuracy: 0.8315 - val_loss: 0.2538 - val_accuracy: 0.8318
Epoch 7/150
375/375 [==============================] - 584s 2s/step - loss: 0.4348 - accuracy: 0.8338 - val_loss: 0.3022 - val_accuracy: 0.8353
Epoc

375/375 [==============================] - 581s 2s/step - loss: 0.2063 - accuracy: 0.8852 - val_loss: 0.1514 - val_accuracy: 0.8856
Epoch 58/150
375/375 [==============================] - 577s 2s/step - loss: 0.2027 - accuracy: 0.8859 - val_loss: 0.1522 - val_accuracy: 0.8863
Epoch 59/150
375/375 [==============================] - 577s 2s/step - loss: 0.2241 - accuracy: 0.8866 - val_loss: 0.1449 - val_accuracy: 0.8869
Epoch 60/150
375/375 [==============================] - 578s 2s/step - loss: 0.1811 - accuracy: 0.8873 - val_loss: 0.1488 - val_accuracy: 0.8876
Epoch 61/150
375/375 [==============================] - 585s 2s/step - loss: 0.1910 - accuracy: 0.8880 - val_loss: 0.1513 - val_accuracy: 0.8884
Epoch 62/150
375/375 [==============================] - 565s 2s/step - loss: 0.1851 - accuracy: 0.8888 - val_loss: 0.1441 - val_accuracy: 0.8891
Epoch 63/150
375/375 [==============================] - 571s 2s/step - loss: 0.1817 - accuracy: 0.8895 - val_loss: 0.1360 - val_accuracy: 0.889

375/375 [==============================] - 572s 2s/step - loss: 0.0863 - accuracy: 0.9184 - val_loss: 0.2353 - val_accuracy: 0.9186
Epoch 114/150
375/375 [==============================] - 574s 2s/step - loss: 0.1024 - accuracy: 0.9188 - val_loss: 0.2407 - val_accuracy: 0.9190
Epoch 115/150
375/375 [==============================] - 576s 2s/step - loss: 0.0941 - accuracy: 0.9192 - val_loss: 0.2456 - val_accuracy: 0.9194
Epoch 116/150
375/375 [==============================] - 580s 2s/step - loss: 0.0808 - accuracy: 0.9196 - val_loss: 0.2389 - val_accuracy: 0.9198
Epoch 117/150
375/375 [==============================] - 586s 2s/step - loss: 0.0808 - accuracy: 0.9200 - val_loss: 0.2502 - val_accuracy: 0.9202
Epoch 118/150
375/375 [==============================] - 578s 2s/step - loss: 0.0879 - accuracy: 0.9204 - val_loss: 0.2531 - val_accuracy: 0.9206
Epoch 119/150
375/375 [==============================] - 579s 2s/step - loss: 0.0774 - accuracy: 0.9209 - val_loss: 0.2520 - val_accuracy:

FileNotFoundError: [Errno 2] No such file or directory: '../results/experiment_results.json'

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored